# Objects - Part I

Objects are bundles of data with some attitude. Individuals like you and me are objects. We carry data: our name, age, gender, credit score, income, traffic tickets, social security number, and even secrets—like a taste for pineapple on pizza.

Our behavior is our attitude. It governs how we share this data with others. We’ll readily answer “What’s your name?” but hesitate before revealing a social security number.

As objects, we use _judgment_ to decide what information to share freely and what to protect. Ultimately, all information can be shared under the right authority. For instance, you wouldn’t disclose your income or SSN to a stranger on the street—but you would share them with a stranger behind a bank’s loan desk when applying for a mortgage.

Objects in programming, however, don’t have judgment. Left on their own, they’ll share _everything_ about themselves.

## Encapsulation

The closest things that objects has to judgement is called _encapsulation_ in object-oriented programming. Encapsulation is the technique of specifying what data and methods an object should expose and how it should share them.

Consider the following simple object that describes a person. (For simplicity, I omit type hints; the full code can be found in [Person_Protected.py](./Person_Protected.py)).

```python
class Person:
    def __init__(self, name, year_born):
        self.name = name
        self.year_born = year_born
        self.__ssn = -1
```

The double underscores in the variable named `__ssn` signify that it's a protected data field. It cannot be divulged without some proper authorization. Double underscores _do not really protect_ data fields. Instead they make them a bit more difficult to access through a technique called _name mangling_. The contents of `__ssn` are still accessible as `_Person.__ssn`. This is demonstrated in [Person_Implementation.py](./Person_Implementation.py).

Encapsulation also works for method names when we do not wish other programs to use them.

Python deliberately does not enforce strict encapsulation. Guido van Rossum, the creator of the language, once remarked "we are all adults here" to justify this design choice. His point was that developers are expected to respect the convention of marking certain fields as private rather than being forced to do so by the interpreter. For this reason, many Python programmers use a single leading underscore to signal a private field, avoiding name mangling altogether. 

The single or double underscore signals encapsulation only when used in the beginning of a name, for example, `__ssn` etc. There are names, in Python, surrounded by double underscores; most notably `__init__`. These are **special methods** supporting integration between an object and the language's data model, i.e. the formal set of rules that defines how data types, objects, and their behaviors are represented and interact within the language. These special methods are colloquially called *magic methods* or *dunder* methods.

In contrast to Python's lax (well, non-existence) enforcement of encapsulation, there are languages like Java that are must stricter. This is demonstrated with three easy-to-follow programs listed here.

- [PersonExposed](./PersonExposed.java) is a `Person`-like object with all its fields exposed and accessible to everyone. There is very little safeguarding from unauthorized access.
- [PersonProtected](./PersonProtected.java) is also a a `Person`-like object with all its fields exposed and accessible to everyone _except_ for the social security number which is declared `private`. This tells the compiler to not allow other programs direct access to `ssn`.
- [PersonImplementation](./PersonImplementation.java) implements two simple objects using the classes above. It demonstrates that direct access to the ssn of the protected object will stop the program from even compiling.

## Self reference

In Python, `self` is not a keyword but a naming convention. It refers to the instance on which a method is called, giving access to the object’s attributes and other methods. When we define a method inside a class, we must explicitly include `self` as the first parameter so that Python passes the current object automatically during a call.

We can use any other valid variable name as the first parameter to reference the object. For example, class `Person` in [Person_Protected.py](./Person_Protected.py) can be rewritten as shown below. The names `sake`, `tofu`, and `nori` replace `self` and the object works just fine.


In [ ]:
class Person:
    def __init__(sake, name: str, year_born: int) -> None:
        sake.name: str = name
        sake.year_born: int = year_born
        sake.__ssn: int = -1

    def set_ssn(tofu, ssn: int) -> None:
        tofu.__ssn = ssn

    def get_ssn(nori) -> int:
        return nori.__ssn


if __name__ == "__main__":
    test = Person("Bob", 1985)
    test.set_ssn(111223333)
    print(test.get_ssn())

While we could technically name the object anything (`this`, `me`, etc.), using `self` is the universally accepted style in Python, making code more readable and consistent across projects.


## A simple data structure

Imagine an array-like object with the following guaranteed behavior: its first element is always the smallest element in the array, no matter how the rest of the elements are organized. Consider for example, the array

```python
underlying: list[int] = [10, 30, 20]
```

It exhibits the desired behavior, as `underlying[0]` is its smallest element. In a typical Python setting, if we added a new element to the array via:

```python
underlying.append(5)
```

the result would be

```
[10, 30, 20, 5]
```

However, in the object we design the result should be

```
[5, 10, 30, 20]
```

This data structure can return only one element at a time. And always that will be the first element, i.e., the smallest value. After each removal, the object should ensure that its first element is still the smallest among its remaining elements, by doing the necessary rearrangements.

This behavior is called a _minimum heap._ It's important because it always takes one step only to return the desired result (ie the smallest element in the data structure).

There is an elegant way to implement a minimum heap. We'll discuss later in the course. For now, we'll look at a naive and far less elegant implementation, shown below.



In [ ]:
class Naive_Min_Heap:
    """A simple, not very efficient, min-heap implementation.
    For simplicity we'll assume the data structure stores integer
    values."""

    # Constants
    _EMPTY: str = "Heap is empty."

    def __init__(self) -> None:
        """Basic constructor"""
        self.underlying: list[int] = []

    def get_smallest(self) -> int:
        """Return the smallest element in the heap. This code
        is extremely naive for educational purposes."""
        if not self.underlying:
            raise IndexError(self._EMPTY)
        # Remove the first element of the underlying array so that we can
        # return it at the end of the method
        result = self.underlying.pop(0)
        # Find the next smallest element in the underlying array but
        # only if the array has two or more elements, using the object's
        # internal method __smallest_index.
        if len(self.underlying) > 1:
            next_smallest_index = self.__smallest_index()
            # Swap smallest item and first item, to ensure that smallest
            # item is always at the front of the array, using the object's
            # internal method __swap.
            self.__swap(0, next_smallest_index)
        # Done!
        return result

    def add(self, value: int) -> None:
        """Add a new value to the heap. Method checks first in case new
        value is smaller than first element in which case it swaps first
        and last elements."""
        self.underlying.append(value)
        # If there are two or more elements and the new value is smaller
        # than the first element, swap the first and last elements.
        if len(self.underlying) > 1 and value < self.underlying[0]:
            self.__swap(0, len(self.underlying) - 1)

    def __swap(self, i: int, j: int) -> None:
        """Internal helper method to swap two elements in the underlying array.
        Method checks i and j first to avoid swapping an element with itself."""
        if i != j:
            temp = self.underlying[i]
            self.underlying[i] = self.underlying[j]
            self.underlying[j] = temp

    def __smallest_index(self) -> int:
        """Internal helper method to find the index of the smallest element
        in the underlying array."""
        if not self.underlying:
            raise IndexError(self._EMPTY)
        smallest_index = 0
        # Linear search to find the position of the smallest value
        for i in range(1, len(self.underlying)):
            if self.underlying[i] < self.underlying[smallest_index]:
                smallest_index = i
        return smallest_index


# Quick demo
if __name__ == "__main__":
    # Create a new Naive_Min_Heap instance
    heap = Naive_Min_Heap()

    # Add some elements
    heap.add(5)
    heap.add(3)
    heap.add(8)

    # Get the smallest element
    print(heap.get_smallest())  # Should print 3
    print(heap.get_smallest())  # Should print 5
    heap.add(1)
    print(heap.get_smallest())  # Should print 1